1. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát éves bontásban norbert2 azonosítójú ügyfél rendeléseinek értékéről!</span>

1. A lista megfelelően jelölve tartalmazza a végösszeget is!

In [ ]:
SELECT 
    IIF(GROUPING_ID(YEAR(r.REND_DATUM)) = 1, 
        'Összesen',
        CAST(YEAR(r.REND_DATUM) AS nvarchar(4))),
    SUM(rt.EGYSEGAR*rt.MENNYISEG) --rendelés értéke, nem számoljuk mert agregál
FROM Rendeles r
        JOIN Rendeles_tetel rt ON r.sorszam = rt.sorszam
WHERE LOGIN = 'norbert2'
GROUP BY ROLLUP(YEAR(r.REND_DATUM))

In [ ]:
SELECT *
FROM Rendeles

SELECT *
FROM Rendeles_tetel

2. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát szállítási dátumonként, azon belül szállítási módonként az egyes rendelések összmennyiségéről!</span>

1. Csak azokat a termékeket vegyük figyelembe, amelyek mennyiségi egysége db!
2. A listát szűrjük úgy, hogy az csak a részösszeg sorokat és a végösszeget tartalmazza!

In [ ]:
SELECT
    r.SZALL_DATUM,
    r.SZALL_MOD,
    SUM(rt.MENNYISEG)
FROM Rendeles r JOIN Rendeles_tetel rt ON r.SORSZAM = rt.SORSZAM
                JOIN Termek t ON rt.TERMEKKOD = t.TERMEKKOD
WHERE t.MEGYS = 'db'
GROUP BY ROLLUP(r.SZALL_DATUM, r.SZALL_MOD)
HAVING r.SZALL_MOD IS NULL OR r.SZALL_DATUM IS NULL
--Having grouping_id(r.szall_datum, r.szall_mod) > 0

```
3. Hány olyan ügyfél van, aki még nem rendelt semmit?
```

1. Csoportosítsuk őket nem szerint, azon belül életkor szerint!
2. A lista tartalmazza a részösszegeket és a végösszeget is!

![](https://moodle.uni-corvinus.hu/theme/image.php/adaptable/atto_h5p/1709548862/icon)

4. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a megrendelt termékek legkisebb és legnagyobb egységáráról szállítási dátum, azon belül szállítási mód szerinti bontásban!</span>

1. A lista csak a 2015 májusi szállításokat tartalmazza!
2. Jelenítsük meg a részösszegeket és a végösszeget is!

5. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk csoportot a termékek listaára alapján a következők szerint:&nbsp;<br></span> <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Az "olcsó" termékek legyenek azok, amelyek listaára 3000 alatt van.<br>&nbsp;A "drága" termékek legyenek az 5000 felettiek, a többi legyen "közepes".</span>

1. Listázzuk az egyes csoportokat, és a csoportokba tartozó termékek darabszámát!
2. A lista jelenítse meg a végösszeget is!

6. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát az ügyfelek adatairól név szerinti sorrendben.</span>

1. Minden sorban jelenjen meg a sorrend szerint előző, illetve következő ügyfél neve is.
2. Ha nincs előző vagy következő ügyfél, akkor a 'Nincs' jelenjen meg!

In [ ]:
SELECT
    NEV AS 'név',
    LAG(NEV, 1, 'Nincs előző') OVER(ORDER BY NEV) AS 'előtte', --előtte lévő érték, mennyivel előtte, opcionális: ne null legyen ha nincs érték
    LEAD(NEV, 1, 'Nincs következő') OVER(ORDER BY NEV) AS 'utána' --utána lévő érték
FROM Ugyfel
GROUP BY NEV

7. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk lekérdezést, amely megmutatja, hogy melyik termékkategóriába hány termék tartozik. A lista a kategória nevét és a darabszámot jelenítse meg.</span>

1. A lista ne tartalmazzon duplikált sorokat.
2. A feladatot partíciók segítségével oldjuk meg!

In [ ]:
SELECT DISTINCT
    tk.KAT_NEV,
    t.MEGYS,
    COUNT(*) /*nullokat is beleszámolja*/ OVER(PARTITION BY tk.KAT_NEV), --csoportosítás szempontja, így bármit beírhatunk a selectbe és a groupby-ba nem
    AVG(t.LISTAAR) OVER(PARTITION BY t.RAKTAR_KOD) --bármennyi írható
FROM Termek t JOIN Termekkategoria tk ON t.KAT_ID = tk.KAT_ID
--GROUP BY tk.KAT_NEV

8. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelési tételekről. Az egyes rendelési tételeket termékenként soroljuk be 4 osztályba a rendelés mennyisége alapján. Jelenítsük meg ezt az információt is egy új oszlopban, az oszlop neve legyen 'Mennyiségi kategória'.</span>

1. A lista csak a 100 Ft feletti egységárú rendelési tételeket vegye figyelembe!

In [ ]:
SELECT *,
    NTILE(4) OVER(PARTITION BY EGYSEGAR ORDER BY MENNYISEG) AS 'Mennyiségi kategória' --A partíció elemeit adott számú osztályba sorolja a megadott szempont és sorrend alapján
FROM Rendeles_tetel
WHERE EGYSEGAR > '100'

9. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Listázzuk a termékek kódját, megnevezését, kategóriájának nevét, és listaárát.</span>

1. A listát egészítsük ki két új oszloppal, amelyek a kategória legolcsóbb, illetve legdrágább termékének árát tartalmazzák.
2. A két új oszlop létrehozásánál partíciókkal dolgozzunk!

In [50]:
SELECT --min/max vagy first value/last value
    t.TERMEKKOD,
    t.MEGNEVEZES,
    tk.KAT_NEV,
    t.LISTAAR,
    --FIRST_VALUE(t.LISTAAR) OVER (PARTITION BY tk.KAT_NEV ORDER BY t.LISTAAR)
    MIN(t.LISTAAR) OVER(PARTITION BY tk.KAT_ID) AS 'MIN', --nem muszáj sorrend, anélkül is megvan a min és max
    MAX(t.LISTAAR) OVER(PARTITION BY tk.KAT_ID) AS 'MAX'
FROM Termek t JOIN Termekkategoria tk ON t.KAT_ID = tk.KAT_ID

(551 rows affected)

Total execution time: 00:00:00.014

TERMEKKOD,MEGNEVEZES,KAT_NEV,LISTAAR,MIN,MAX
08070472T,A föld amelyen élünk - Európa földrajza,könyv,1089,214,1089
08070473T,A föld amelyen élünk - Távoli földrészek,könyv,1089,214,1089
08070474T,A föld amelyen élünk - Hazánk földrajza,könyv,1089,214,1089
08070479T,Fizika,könyv,674,214,1089
08070480T,Fizika 13 éveseknek,könyv,308,214,1089
08070489T,RAMba zárt világ,könyv,868,214,1089
08070490T,Szövegszerkesztés,könyv,691,214,1089
08070491T,Szám.tech. kicsiknek,könyv,214,214,1089
08070492T,Informatika,könyv,735,214,1089
08070493T,Érettségi felv. fel. Matematika,könyv,720,214,1089


10. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelésekről. A lista legyen rendezve ügyfelenként (LOGIN), azon belül a rendelés dátuma szerint. A listához készítsünk sorszámozást is. A sorszám a következő formában jelenjen meg: sorszám_év_login. Pl: 1_2015_adam1</span>

1. A számozás login-onként, azon belül rendelési évenként kezdődjön újra.
2. A sorszám oszlop neve legyen Azonosító

11. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a termékek adatairól listaár szerint növekvő sorrendben! A lista jelenítse meg két új oszlopban a sorrend szerint előző, illetve következő termék listaárát is a termék saját kategóriájában és raktárában!</span>

1. Ahol nincs előző vagy következő érték, ott 0 jelenjen meg!
2. Az oszlopokat nevezzük el értelemszerűen!

12. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Listázzuk a termékek kódját, nevét és listaárát listaár szerinti sorrendben!</span>

1. Vegyünk fel egy új oszlopot Mozgóátlag néven, amely minden esetben az aktuális termék, az előző, és a következő termék átlagárát tartalmazza!
2. A mozgóátlagot kerekítsük két tizedesre!

In [ ]:
SELECT
    TERMEKKOD,
    MEGNEVEZES,
    LISTAAR,
    --mozgóátlag
    --ötlet_1: (LAG(LISTAAR) + LISTAAR + LEAD(LISTAAR)...)/3 DE ha 5-tel arrább kell már offos
    AVG(LISTAAR) OVER(ORDER BY LISTAAR ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING )--ablak méretét meg tudom adni nagyobb értéknél
FROM Termek

13. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát, amely a rendelések sorszámát és a rendelés értékét tartalmazza. A listát egészítsük ki egy új oszloppal, amely minden rendelés esetén addigi rendelések összegét tartalmazza (az aktuálisat is beleértve)!</span>

1. A listát rendezzük sorszám szerint növekvő sorrendbe.
2. A lista ne tartalmazzon duplikált sorokat!

14. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a termékek kódjáról, nevéről, kategória azonosítójáról, raktár azonosítójáról és listaáráról, valamint a termék adott szempontok szerinti rangsorokban elfoglalt helyezéseiről. (Szempontonként külön oszlopban, a helyezéseknél növekvő sorrendet feltételezve). A szempontok a következők legyenek: listaár, kategória szerinti listaár, és raktárkód szerinti listaár.</span>

1. Az oszlopokat nevezzük el értelemszerűen.
2. A helyezések egyenlőség esetén "sűrűn" kövessék egymást.
3. A lista legyen rendezett kategória azonosító, azon belül listaár szerint!

15. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelési tételekről, amely minden sor esetén göngyölítve tartalmazza az ügyfél adott rendelési tételig meglévő rendelési tételeinek összértékét!</span>

1. Az új oszlop neve legyen Eddigi rendelési tételek összértéke!
2. Az ügyfél neve is jelenjen meg!